# Getting the US Dollar exchange rate reports

The first step is to get the US Dollar exchange rate reporst that happens throwout the day. There are three types of reports
- Abertura(opening)
- Intermediary(Intermediário)
- Closing(Fechamento)

We are getting the dataset from the API: [Banco Central do Brasil Dollar API](https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/aplicacao#!/)



In [2]:
import requests
import numpy as np

from pandas.io.json import json_normalize

request = requests.get("https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoMoedaPeriodo(moeda=@moeda,dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)?@moeda='USD'&@dataInicial='07-01-2018'&@dataFinalCotacao='08-29-2019'&$format=json&$select=cotacaoCompra,dataHoraCotacao,tipoBoletim")

dolar_rates_json = request.json()

dolar_rates = json_normalize(dolar_rates_json['value'])

dolar_rates.tail(5)

,cotacaoCompra,dataHoraCotacao,tipoBoletim
1454,4.1694,2019-08-29 10:09:17.705,Abertura
1455,4.1696,2019-08-29 11:10:17.541,Intermediário
1456,4.1691,2019-08-29 12:11:17.181,Intermediário
1457,4.1616,2019-08-29 13:09:45.961,Intermediário
1458,4.1674,2019-08-29 13:09:45.964,Fechamento


# Translatting the dataframe to english

Simply translatting the portuguese words to english.

In [3]:
dolar_rates.columns = ['usd_quotation', 'quote_timestamp', 'report_type']
dolar_rates = dolar_rates.replace('Abertura', 'opening')
dolar_rates = dolar_rates.replace('Intermediário', 'intermediary')
dolar_rates = dolar_rates.replace('Fechamento', 'closing')

dolar_rates.tail(5)

,usd_quotation,quote_timestamp,report_type
1454,4.1694,2019-08-29 10:09:17.705,opening
1455,4.1696,2019-08-29 11:10:17.541,intermediary
1456,4.1691,2019-08-29 12:11:17.181,intermediary
1457,4.1616,2019-08-29 13:09:45.961,intermediary
1458,4.1674,2019-08-29 13:09:45.964,closing


# Categorizing US Dollar ratings

Now we are categorizing each rating, giving them one of three classes:

1. "UP" - If the USD_QUOTATION is greater than the previous value
2. "STEADY"- If the USD_QUOTATION is equal to the previous value
3. "DOWN" - If the USD_QUOTATION is smaller than the previous value

In [4]:
dolar_rates['variation'] = np.where(dolar_rates['usd_quotation'].diff() > 0, 'up', 
                           np.where(dolar_rates['usd_quotation'].diff() < 0, 'down', 'steady'))

dolar_rates = dolar_rates.drop(['usd_quotation', 'report_type'], axis=1)

dolar_rates.tail(5)

,quote_timestamp,variation
1454,2019-08-29 10:09:17.705,up
1455,2019-08-29 11:10:17.541,up
1456,2019-08-29 12:11:17.181,down
1457,2019-08-29 13:09:45.961,down
1458,2019-08-29 13:09:45.964,up


In [5]:
dolar_rates.to_csv('../datasets/pre-processed/dolar_rates.csv', index= False, encoding='utf-8')